Helper functions to convert the problem into the appropriate Hamiltonian form.


In [1]:
from pyqubo import Spin, Binary, Array
import math
import numpy as np


def basis_to_gram_matrix(basis):
    if not isinstance(basis, np.ndarray):
        raise ValueError("Basis must be a numpy array")
    
    return basis @ basis.T


def gram_to_quadratic_form(gram_matrix):
    x, y = gram_matrix.shape
    if x != y:
        raise ValueError("Gram matrix must be square")
    
    # Diagonal terms
    diag = {}
    for i in range(x):
        diag[i] = gram_matrix[i, i]
    
    # Off-diagonal terms
    off_diag = {}
    for i in range(x):
        for j in range(i + 1, y):
            off_diag[(i, j)] = gram_matrix[i, j] + gram_matrix[j, i]
    
    return diag, off_diag


def substitution_array(variable_name, num_variables, bounds):
    # Handle bounds input
    if isinstance(bounds, (int, float)):
        bounds = [bounds] * num_variables
    elif len(bounds) != num_variables:
        raise ValueError("Bounds list must match number of variables")
    
    substitutions = []
    
    for i, bound in enumerate(bounds):
        num_bits = math.floor(math.log2(2 * bound)) + 1
        
        # Create binary array for this variable
        binary_vars = Array.create(f"{variable_name}_{i}", shape=num_bits, vartype='BINARY')
        
        # Coefficients for the encoding
        coeffs = [2**j for j in range(num_bits - 1)]
        last_coeff = 2 * bound + 1 - 2**math.floor(math.log2(2 * bound))
        coeffs.append(last_coeff)
        
        # Build the substitution expression
        sub = -bound
        for j, coeff in enumerate(coeffs):
            sub += coeff * binary_vars[j]
        
        substitutions.append(sub)
    print(substitutions)
    return substitutions

def substitution_array_for_nonzero(variable_name, num_variables, bounds):
    if isinstance(bounds, (int, float)):
        bounds = [bounds] * num_variables
    elif len(bounds) != num_variables:
        raise ValueError("Bounds list must match number of variables")
    
    substitutions = []
    
    for i, bound in enumerate(bounds):
        # Calculate number of bits needed
        num_bits = math.floor(math.log2(bound - 1)) + 1
        print(f"Nium bits = {num_bits} for bound {bound}")
        
        # Create binary array for this variable
        binary_vars = Array.create(f"{variable_name}_{i}", shape=num_bits + 2, vartype='BINARY')
        
        # x_i = -a_i + ξ_i + ω_i(a + 1) + Σ(2^j * x_ij) + (a_i - 2^⌊log(a_i-1)⌋) * x_i,⌊log(a-1)⌋
        
        # Build the substitution expression
        sub = -bound  # -a_i term
        
        # ξ_i term (first binary variable)
        sub += binary_vars[0]
        
        # ω_i(a + 1) term (second binary variable)
        sub += binary_vars[1] * (bound + 1)
        
        # Σ(2^j * x_ij) terms for j from 0 to ⌊log(a_i-1)⌋-1
        for j in range(num_bits-1):
            sub += (2**j) * binary_vars[j + 2]
        
        # Final term: (a_i - 2^⌊log(a_i-1)⌋) * x_i,⌊log(a-1)⌋
        if num_bits > 0:
            final_coeff = bound - (2**math.floor(math.log2(bound - 1)))
            sub += final_coeff * binary_vars[num_bits + 1]
            print(f"Appending substitution for {final_coeff} * {binary_vars[num_bits + 1]}")
        substitutions.append(sub)
    print(substitutions)
    return substitutions

def gram_to_hamiltonian(gram_matrix, bounds, variable_name='x'):
    n = gram_matrix.shape[0]
    
    # Convert Gram matrix to quadratic form
    diagonal_terms, off_diagonal_terms = gram_to_quadratic_form(gram_matrix)
    
    # Create substitutions - special non-zero substitution only for x_0
    substitutions = []
    
    for i in range(n):
        # if False: USE THIS IF YOU WANT TO USE THE NORMAL SUBSTITUTION
        if i == 0:
            # Use special non-zero substitution for x_0
            # Extract the bound for this variable and ensure it's a scalar
            if isinstance(bounds, (list, np.ndarray)) and len(bounds) > 1:
                bound_for_i = int(bounds[0])  # Convert to Python int
            else:
                bound_for_i = int(bounds) if isinstance(bounds, np.ndarray) else bounds
            special_sub = substitution_array_for_nonzero(f"{variable_name}_control", 1, bound_for_i)
            substitutions.append(special_sub[0])
        else:
            # Use regular substitution for all other variables
            # Extract the bound for this variable and ensure it's a scalar
            if isinstance(bounds, (list, np.ndarray)) and len(bounds) > 1:
                bound_for_i = int(bounds[i])  # Convert to Python int
            else:
                bound_for_i = int(bounds) if isinstance(bounds, np.ndarray) else bounds
            # FIX: Include the variable index in the variable name
            regular_sub = substitution_array(f"{variable_name}_{i}", 1, bound_for_i)
            substitutions.append(regular_sub[0])
    
    # Build the Hamiltonian
    hamiltonian = 0
    
    # Add penalty for the first binary variable of x_0 (x_control_0) = 1
    # This corresponds to the ξ_0 term in the non-zero encoding
    if len(substitutions) > 0:  # Ensure x_0 exists
        # Create the first binary variable for the penalty
        penalty_var = Binary(f"{variable_name}_control_0[0]")  # First binary var of first special variable
        hamiltonian += 40 * penalty_var  # Penalty of 5 when this binary variable = 1
        # testing_var = Binary(f"{variable_name}_0[0]")  # Second binary var of first special variable
        ## CHANGE THIS TO USE PENALITY VAR, UNCOMMENT THE ABOVE LINES
    
    # Add diagonal terms (quadratic terms like ax²)
    for i, coeff in diagonal_terms.items():
        hamiltonian += coeff * (substitutions[i] ** 2)
    
    # Add off-diagonal terms (cross terms like bxy)
    for (i, j), coeff in off_diagonal_terms.items():
        hamiltonian += coeff * substitutions[i] * substitutions[j]
    
    
    
    
    # Compile the model
    model = hamiltonian.compile()
    return model


def generate_qubo_from_gram_matrix_array(gram_matrix, bounds, variable_name='x'):
    model = gram_to_hamiltonian(gram_matrix, bounds, variable_name)
    qubo_dict, offset = model.to_qubo()
    
    return qubo_dict, offset
def generate_ising_from_gram_matrix_array(gram_matrix, bounds, variable_name='x'):
    model = gram_to_hamiltonian(gram_matrix, bounds, variable_name)
    linear, quadratic, offset = model.to_ising()
    
    return linear , quadratic, offset

def basis_to_ising(basis, bounds, variable_name='x'):
    gram_matrix = basis_to_gram_matrix(basis)
    return generate_ising_from_gram_matrix_array(gram_matrix, bounds, variable_name)

CONVERT TO APPROPRIATE QISKIT FORM

In [2]:
import numpy as np
from qiskit.quantum_info import SparsePauliOp
import re

def ising_to_sparse_pauli_op(linear_terms, quadratic_terms, offset):
    # Extract all variable names and map them to qubit indices
    all_vars = set()
    for var in linear_terms.keys():
        all_vars.add(var)
    for var_pair in quadratic_terms.keys():
        all_vars.update(var_pair)
    
    # Sort variables to ensure consistent ordering
    sorted_vars = sorted(list(all_vars))
    
    print("Sorted variables:", sorted_vars)
    
    # Create mapping from variable name to qubit index
    var_to_qubit = {var: i for i, var in enumerate(sorted_vars)}
    n_qubits = len(sorted_vars)

    
    pauli_list = []
    
    # Add constant term (identity operator)
    if offset != 0:
        identity_string = 'I' * n_qubits
        pauli_list.append([identity_string, offset])
    
    # Add linear terms (single Z operators)
    for var, coeff in linear_terms.items():
        pauli_string = ['I'] * n_qubits
        qubit_idx = var_to_qubit[var]
        pauli_string[qubit_idx] = 'Z'
        pauli_list.append([''.join(pauli_string), coeff])
    
    # Add quadratic terms (ZZ operators)
    for (var1, var2), coeff in quadratic_terms.items():
        pauli_string = ['I'] * n_qubits
        qubit_idx1 = var_to_qubit[var1]
        qubit_idx2 = var_to_qubit[var2]
        pauli_string[qubit_idx1] = 'Z'
        pauli_string[qubit_idx2] = 'Z'
        pauli_list.append([''.join(pauli_string), coeff])
    
    return SparsePauliOp.from_list(pauli_list)

basis = np.array([[-5, -2, 2], [-2, 5, 7], [6, -1, 6]])
bound = np.array([3,3,3])
ising = basis_to_ising(basis, bound)


print("Raw ising output:")
print(ising)
print()

# Unpack the result
linear_terms, quadratic_terms, offset = ising

print("Linear terms:", linear_terms)
print("Quadratic terms:", list(quadratic_terms.keys()))  # Show first 5 keys
print("Offset:", offset)
print()

# Convert to SparsePauliOp
ising_op =  ising_to_sparse_pauli_op(linear_terms, quadratic_terms, offset)
print("SparsePauliOp:")
print(ising_op)

Nium bits = 2 for bound 3
Appending substitution for 1 * Binary('x_control_0[3]')
[((1.000000 * Binary('x_control_0[3]')) + (1.000000 * Binary('x_control_0[2]')) + (Binary('x_control_0[1]') * 4.000000) + -3.000000 + Binary('x_control_0[0]'))]
[((3.000000 * Binary('x_1_0[2]')) + (2.000000 * Binary('x_1_0[1]')) + -3.000000 + (1.000000 * Binary('x_1_0[0]')))]
[((3.000000 * Binary('x_2_0[2]')) + (2.000000 * Binary('x_2_0[1]')) + -3.000000 + (1.000000 * Binary('x_2_0[0]')))]
Raw ising output:
({'x_2_0[1]': -16.0, 'x_1_0[1]': 14.0, 'x_2_0[0]': -8.0, 'x_1_0[0]': 7.0, 'x_1_0[2]': 21.0, 'x_2_0[2]': -24.0, 'x_control_0[0]': 36.5, 'x_control_0[1]': 66.0, 'x_control_0[2]': 16.5, 'x_control_0[3]': 16.5}, {('x_1_0[0]', 'x_1_0[2]'): 117.0, ('x_1_0[1]', 'x_control_0[1]'): 56.0, ('x_1_0[0]', 'x_2_0[0]'): 12.5, ('x_1_0[0]', 'x_2_0[1]'): 25.0, ('x_1_0[0]', 'x_1_0[1]'): 78.0, ('x_1_0[0]', 'x_2_0[2]'): 37.5, ('x_1_0[0]', 'x_control_0[0]'): 7.0, ('x_1_0[0]', 'x_control_0[1]'): 28.0, ('x_1_0[1]', 'x_1_0[2]')

CVar Optimization

In [3]:
import numpy as np
from qiskit.circuit.library import QAOAAnsatz #Defines the starting point for QAOA circuits
from qiskit.primitives import StatevectorEstimator, StatevectorSampler #Whats the difference?
from scipy.optimize import minimize #COBYLA and Powell optimization methods
import matplotlib.pyplot as plt #plotting results

def create_qaoa_circuit(cost_operator, reps=1):

    return QAOAAnsatz(
        cost_operator=cost_operator,
        reps=reps,
        flatten=True  # IMPORTANT for performance!
    )

def create_sampling_circuit(qaoa_ansatz, parameters):
    """Create circuit with measurements for final sampling"""
    bound_circuit = qaoa_ansatz.assign_parameters(parameters)
    bound_circuit.measure_all()
    return bound_circuit


def objective_function_fixed(parameters, qaoa_ansatz, cost_operator, use_cvar=True, alpha=0.25, shots=1024):

    if not use_cvar:
        # Original expectation value approach
        estimator = StatevectorEstimator()
        
        # Handle layout for hardware compatibility
        if hasattr(qaoa_ansatz, 'layout') and qaoa_ansatz.layout is not None:
            isa_hamiltonian = cost_operator.apply_layout(qaoa_ansatz.layout)
        else:
            isa_hamiltonian = cost_operator
        
        # Use the pub format
        pub = (qaoa_ansatz, isa_hamiltonian, parameters)
        job = estimator.run([pub])
        results = job.result()[0]
        
        return float(results.data.evs)
    
    else:
        # CVaR approach: sample and focus on best solutions
        
        # Create sampling circuit with measurements
        bound_circuit = qaoa_ansatz.assign_parameters(parameters)
        bound_circuit.measure_all()
        
        # Sample from the quantum state
        sampler = StatevectorSampler()
        job = sampler.run([bound_circuit], shots=shots)
        counts = job.result()[0].data.meas.get_counts()
        
        # Evaluate costs for all sampled bitstrings
        costs = []
        for bitstring, count in counts.items():
            cost = evaluate_bitstring_cost(bitstring, cost_operator)
            costs.extend([cost] * count)  # Repeat cost for each occurrence
        
        # Convert to numpy array and sort
        costs = np.array(costs)
        costs_sorted = np.sort(costs)
        
        # Calculate cutoff for best α fraction
        cutoff_index = int(alpha * len(costs))
        if cutoff_index == 0:
            cutoff_index = 1
        
        # CVaR: average of the best α fraction of solutions
        cvar_value = np.mean(costs_sorted[:cutoff_index])
        
        return float(cvar_value)

    
    
def run_qaoa_optimization(cost_operator, reps=1, max_iterations=500, use_cvar=True, cvar_alpha=0.25, cvar_shots=1024):
    """
    MODIFIED: Added CVaR parameters
    Complete QAOA optimization workflow with CVaR support
    
    Args:
        cost_operator: The cost Hamiltonian
        reps: Number of QAOA layers
        max_iterations: Maximum optimization iterations
        use_cvar: If True, use CVaR objective function
        cvar_alpha: CVaR risk level (0.1 = best 10%, 0.25 = best 25%)
        cvar_shots: Number of shots for CVaR sampling
    """
    
    # Create QAOA ansatz (using your original simple version)
    qaoa_ansatz = create_qaoa_circuit(cost_operator, reps=reps)
    
    print(f"QAOA circuit has {qaoa_ansatz.num_parameters} parameters")
    print(f"Circuit depth: {qaoa_ansatz.depth()}")
    print(f"Number of qubits: {qaoa_ansatz.num_qubits}")
    
    if use_cvar:
        print(f"Using CVaR optimization:")
        print(f"  - Risk level (alpha): {cvar_alpha} (focusing on best {cvar_alpha*100:.1f}% of solutions)")
        print(f"  - Shots per evaluation: {cvar_shots}")
    else:
        print("Using standard expectation value optimization")
    
    # Better parameter initialization for QAOA
    np.random.seed(69)
    initial_params = np.random.uniform(0, 0.5, qaoa_ansatz.num_parameters)
    print(f"Initial parameters: {initial_params}")
    
    # Test objective function
    print("Testing objective function...")
    initial_energy = objective_function_fixed(initial_params, qaoa_ansatz, cost_operator, 
                                            use_cvar=use_cvar, alpha=cvar_alpha, shots=cvar_shots)
    print(f"Initial {'CVaR' if use_cvar else 'energy'}: {initial_energy:.3f}")
    
    # Run optimization
    print("Starting optimization...")
    
    # Try COBYLA first (good for QAOA)
    result = minimize(
        objective_function_fixed,
        initial_params,
        args=(qaoa_ansatz, cost_operator, use_cvar, cvar_alpha, cvar_shots),
        method='COBYLA',
        options={'maxiter': max_iterations, 'disp': True}
    )
    
    # If COBYLA didn't converge, try Powell
    if not result.success and result.nfev >= max_iterations * 0.8:
        print("\\nCOBYLA hit max iterations, trying Powell method...")
        result = minimize(
            objective_function_fixed,
            result.x,  # Start from COBYLA result
            args=(qaoa_ansatz, cost_operator, use_cvar, cvar_alpha, cvar_shots),
            method='Powell',
            options={'maxiter': max_iterations//2, 'disp': True}
        )
    
    print(f"\\nOptimization completed!")
    print(f"Success: {result.success}")
    print(f"Optimal {'CVaR' if use_cvar else 'energy'}: {result.fun:.3f}")
    print(f"{'CVaR' if use_cvar else 'Energy'} improvement: {initial_energy - result.fun:.3f}")
    print(f"Message: {result.message}")
    
    return result, qaoa_ansatz


def sample_final_results(qaoa_ansatz, optimal_params, shots=1000):
    """Sample from the optimized quantum state"""
    
    # Create sampling circuit with measurements
    sampling_circuit = create_sampling_circuit(qaoa_ansatz, optimal_params)
    
    # Use StatevectorSampler for sampling
    sampler = StatevectorSampler()
    job = sampler.run([sampling_circuit], shots=shots)
    counts = job.result()[0].data.meas.get_counts()
    
    return counts

def evaluate_bitstring_cost(bitstring, cost_operator):
    """Evaluate the actual cost of a solution bitstring"""
    z_values = [1 if bit == '0' else -1 for bit in bitstring]
    
    cost = 0
    for pauli_string, coeff in zip(cost_operator.paulis, cost_operator.coeffs):
        term_value = 1
        for i, pauli in enumerate(str(pauli_string)):
            if pauli == 'Z':
                term_value *= z_values[i]
        cost += coeff.real * term_value
    
    return cost

# Example usage with your ising_op from previous cells
def complete_qaoa_workflow(ising_op, reps=1, use_cvar=True, cvar_alpha=0.25, cvar_shots=1024):
    
    print(f"  - QAOA layers (reps): {reps}")
    print(f"  - Use CVaR optimization: {use_cvar}")
    if use_cvar:
        print(f"  - CVaR risk level (alpha): {cvar_alpha}")
        print(f"  - CVaR shots per evaluation: {cvar_shots}")
    print()
    
    # Run optimization
    result, qaoa_ansatz = run_qaoa_optimization(ising_op, reps=reps, max_iterations=500,
                                              use_cvar=use_cvar,
                                              cvar_alpha=cvar_alpha,
                                              cvar_shots=cvar_shots)
    
    print(f"\\nProceeding with final sampling...")
    
    # Sample from optimized state (always use more shots for final analysis)
    print("Sampling from optimized state...")
    final_shots = max(cvar_shots, 1000)  # Use at least 1000 shots for final sampling
    counts = sample_final_results(qaoa_ansatz, result.x, shots=final_shots)
    
    # Analyze top solutions
    print(f"\\nTop 10 most frequent solutions (from {final_shots} shots):")
    sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    
    for i, (bitstring, count) in enumerate(sorted_counts[:10]):
        probability = count / final_shots
        actual_cost = evaluate_bitstring_cost(bitstring, ising_op)
        print(f"{i+1:2d}. {bitstring}: {count:3d} counts ({probability:.1%}), cost = {actual_cost:.1f}")
    
    # Find the best solution by actual cost
    print("\\nBest solutions by actual cost:")
    cost_sorted = [(bs, cnt, evaluate_bitstring_cost(bs, ising_op)) 
                   for bs, cnt in sorted_counts]
    cost_sorted.sort(key=lambda x: x[2])  # Sort by cost
    
    for i, (bitstring, count, cost) in enumerate(cost_sorted[:30]):
        probability = count / final_shots
        print(f"{i+1}. {bitstring}: cost = {cost:.1f}, frequency = {count} ({probability:.1%})")
    
    # CVaR analysis
    if use_cvar:
        print(f"CVAR ANALYSIS")
        
        # Calculate actual CVaR from final sampling
        all_costs = []
        for bitstring, count in sorted_counts:
            cost = evaluate_bitstring_cost(bitstring, ising_op)
            all_costs.extend([cost] * count)
        
        all_costs = np.array(all_costs)
        all_costs_sorted = np.sort(all_costs)
        
        # CVaR for different risk levels
        for alpha in [0.1, 0.25, 0.5]:
            cutoff = int(alpha * len(all_costs))
            if cutoff == 0:
                cutoff = 1
            cvar_val = np.mean(all_costs_sorted[:cutoff])
            print(f"CVaR (α={alpha}): {cvar_val:.1f} (best {alpha*100:.0f}% of solutions)")
        
        print(f"Mean cost: {np.mean(all_costs):.1f}")
        print(f"Best cost found: {np.min(all_costs):.1f}")
        print(f"Worst cost found: {np.max(all_costs):.1f}")
    
    return result, sorted_counts
# Run the complete workflow
print("===time to run programme===")
#optimization_result, top_solutions = complete_qaoa_workflow(ising_op, reps=5)
# Standard CVaR optimization (recommended)
# Test CVaR with your existing setup
optimization_result, top_solutions = complete_qaoa_workflow(
    ising_op, 
    reps=5,
    use_cvar=True,           # Enable CVaR
    cvar_alpha=0.25,         # Focus on best 25%
    cvar_shots=1024          # Shots per evaluation
)

===time to run programme===
  - QAOA layers (reps): 5
  - Use CVaR optimization: True
  - CVaR risk level (alpha): 0.25
  - CVaR shots per evaluation: 1024

QAOA circuit has 10 parameters
Circuit depth: 119
Number of qubits: 10
Using CVaR optimization:
  - Risk level (alpha): 0.25 (focusing on best 25.0% of solutions)
  - Shots per evaluation: 1024
Initial parameters: [0.14812458 0.40453386 0.17512626 0.39470463 0.28067449 0.12679448
 0.05248854 0.02923036 0.33664619 0.34891483]
Testing objective function...
Initial CVaR: 204.035
Starting optimization...
\nOptimization completed!
   Normal return from subroutine COBYLA

   NFVALS =   83   F = 1.731680E+02    MAXCV = 0.000000E+00
   X = 1.072839E+00   1.660990E-01   1.751492E-01   1.394731E+00   2.805846E-01
       1.268617E-01   5.243900E-02   2.925819E-02   3.366606E-01   3.488909E-01

Success: True
Optimal CVaR: 173.168
CVaR improvement: 30.867
Message: Optimization terminated successfully.
\nProceeding with final sampling...
Samplin